In [1]:
import glob
import os
from osgeo import gdal, osr
import numpy as np
import pandas as pd
import rasterio
from rasterio.plot import show

In [2]:
# H5 Sierra Fraction Data Downloaded from here: https://snow.ucsb.edu/products/SPIRES/Sierra/
# Glob together all of the Snow Fraction datasets.
snow_ds = glob.glob('*.h5')
snow_ds

['Sierra2001.h5',
 'Sierra2002.h5',
 'Sierra2003.h5',
 'Sierra2004.h5',
 'Sierra2005.h5',
 'Sierra2006.h5',
 'Sierra2007.h5',
 'Sierra2008.h5',
 'Sierra2009.h5',
 'Sierra2010.h5',
 'Sierra2011.h5',
 'Sierra2012.h5',
 'Sierra2013.h5',
 'Sierra2014.h5',
 'Sierra2015.h5',
 'Sierra2016.h5',
 'Sierra2017.h5',
 'Sierra2018.h5',
 'Sierra2019.h5']

In [17]:
# Get subdatasets of first snow fraction dataset ('Sierra2001.h5').
datasets = gdal.Open(snow_ds[0], gdal.GA_ReadOnly).GetSubDatasets()

#(sds[3] is to choose the 4th dataset in the subdirectory (i.e., snow fraction). 
#The second bracket [0] is needed to open the dataset.
snow_data = gdal.Open(datasets[3][0])

#Changes the selected dataset into an array.
snow_data_array = snow_data.ReadAsArray()

#Converts the variables to 'float' to allow us to convert NA values (255) to nans
#We also convert 0s to nans so that when plotted on base map, only areas where data is present are shown
snow_data_float=snow_data_array.astype('float')
snow_data_float[snow_data_float == 255] = np.nan
# Review whether to make snow 0 or na 
snow_data_float[snow_data_float == 0] = np.nan
snow_data_transposed = np.transpose(snow_data_float)

#Sample data 
snow_data_subset = snow_data_transposed[:,:,160]
snow_data_permuted = np.transpose(snow_data_float, (0, 2, 1))

In [18]:
# x dimension of array
xdim = snow_data_array.shape[1]
# y dimension of array
ydim = snow_data_array.shape[2]
# Projection data of sample GeoTiff
projection = 'PROJCS["Albers Conical Equal Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["standard_parallel_1",34],PARAMETER["standard_parallel_2",40.5],PARAMETER["false_easting",0],PARAMETER["false_northing",-4000000],UNIT["meters",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'
# transformation data of array
# Pull refrencing matrix from h5 file.
ref_matrix_meta = snow_data.GetMetadata()['Grid_MODIS_GRID_500m_ReferencingMatrix'].split()
referencing_matrix = [int(ref_matrix_meta[2]), int(ref_matrix_meta[1]), int(ref_matrix_meta[0]), int(ref_matrix_meta[5]), int(ref_matrix_meta[4]), int(ref_matrix_meta[3])]

## Convert Single Single H5 Layer to Geotiff

In [7]:
def SingleGeotiff(raster_name, data, height, width, geotransform, wkt):
    
    driver = gdal.GetDriverByName('GTiff')

    dataset = driver.Create(
        raster_name,
        width,
        height,
        1,
        gdal.GDT_Float32)

    dataset.SetGeoTransform((
     geotransform))

    dataset.SetProjection(wkt)
    dataset.GetRasterBand(1).WriteArray(data)
    dataset.FlushCache()  # Write to disk.
    return dataset, dataset.GetRasterBand(1) 

In [20]:
SingleGeotiff('test_file.tif', snow_data_subset, ydim, xdim, referencing_matrix, projection)

(<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000015DEE2E53F0> >,
 <osgeo.gdal.Band; proxy of <Swig Object of type 'GDALRasterBandShadow *' at 0x0000015DEE2A29C0> >)

## Convert Stacked H5 File into N Single Geotiffs

In [86]:
path = 'tif/2001'
for i in range(len(snow_data_float)):
    dest = (str(i + 1) + '2001.tif') 
    name = os.path.join(path, dest)
    data = snow_data_transposed[:,:,i]
    SingleGeotiff(name, data, ydim, xdim, referencing_matrix, projection)

In [ ]:
path = 'tif/2001'
dest = 'sample'
name = os.path.join(path, dest)
name

'tif/2001\\sample'

## Convert Stacked H5 File to Geotiff

In [7]:
def StackedGeotiff(name, array, geo_transform, projection):
    
    driver = gdal.GetDriverByName('GTiff')

    DataSet = driver.Create(name, array.shape[2], array.shape[1], array.shape[0], gdal.GDT_Float32)
    DataSet.SetGeoTransform(geo_transform)
    DataSet.SetProjection(projection)
    for i, image in enumerate(array, 1):
        DataSet.GetRasterBand(i).WriteArray( image )
    DataSet.FlushCache()
    return name

In [8]:
StackedGeotiff('sample_stack.tif', snow_data_permuted, referencing_matrix, projection)

'sample_stack10.tif'

## Transform Geotiff to WGS84 

In [23]:
# Change the following variables to the file you want to convert (inputfile)
# and what you want to name your output file (outputfile)
inputfile = "test_file.tif"
outputfile = "wgs_test.tif"
#Do not change the following line, it will reproject the geotiff file
ds = gdal.Warp(outputfile, inputfile, dstSRS="+proj=longlat +datum=WGS84 +no_defs", dstNodata = 0)